In [1]:
import requests
import browser_cookie3
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook
import time
import random

In [2]:
def append_data_to_excel(file_name, data):
    try:
        workbook = load_workbook(file_name)
        sheet = workbook.active
    except FileNotFoundError:
        workbook = Workbook()
        sheet = workbook.active
        sheet.append(["ID", "Announced date", "time", "from", "to", "Moving date", "Commission", "Status"])

    sheet.append(data)
    workbook.save(file_name)

In [3]:
headers = {
    "User-Agent": "THIS HAS BEEN REMOVED FOR SECURITY REASONS",
    "Referer": "THIS HAS BEEN REMOVED FOR SECURITY REASONS",
    "Content-Type": "THIS HAS BEEN REMOVED FOR SECURITY REASONS",
}

payload = {
    "action": "send-quote",
    "quote_template": "",
    "is_html": "1",
    "quote_body": """<p><strong>Hi Dear Customer,</strong></p><p>&nbsp;</p><p>Thank you for considering ******* 🚛</p>"""
}

file_name = "Muuttomaailma.xlsx"

In [ ]:
while True:
    data_ids = set()

    cookies = browser_cookie3.chrome()
    url = "https://crm.muuttomaailma.fi/job/index/all"

    response = requests.get(url, cookies=cookies)

    soup = BeautifulSoup(response.content, 'html.parser')
    trs = soup.find_all('tr', {'data-id': True})
    data_ids = {tr['data-id'] for tr in trs}
    data_ids = list(data_ids)

    # Relocations that are beyond my capabilities
    url = "https://crm.muuttomaailma.fi/job/index/intl"
    response = requests.get(url, cookies=cookies)
    soup = BeautifulSoup(response.content, 'html.parser')
    trs = soup.find_all('tr', {'data-id': True})
    data_ids_intl = {tr['data-id'] for tr in trs}
    data_ids_intl = list(data_ids_intl)

    # Piano
    url = "https://crm.muuttomaailma.fi/job/index/piano"
    response = requests.get(url, cookies=cookies)
    soup = BeautifulSoup(response.content, 'html.parser')
    trs = soup.find_all('tr', {'data-id': True})
    data_ids_piano = {tr['data-id'] for tr in trs}
    data_ids_piano = list(data_ids_piano)

    # Remove from the main list
    for i in range(len(data_ids_intl)):
        if data_ids_intl[i] in data_ids:
            data_ids.remove(data_ids_intl[i])

    for i in range(len(data_ids_piano)):
        if data_ids_piano[i] in data_ids:
            data_ids.remove(data_ids_piano[i])

    # Submitting offer for the remaining orders
    for i in range(len(data_ids)):
        rnd = random.randint(5,10)
        time.sleep(rnd)
        url = "https://crm.muuttomaailma.fi/job/view/" + data_ids[i]
        response = requests.get(url, cookies=cookies)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extracting some data from the order to save on local drive 
        columns = soup.find_all('div', class_='four wide column')
        for column in columns:
            text = column.get_text()
            if 'km' in text:
                distance_ =(text.split('km')[0].strip())[-2:].strip()
            if 'Mistä' in text:
                from_ = text.split('Mistä')[1].strip()
            if 'Mihin' in text:
                where_ = text.split('Mihin')[1].strip()
            if 'Toivottu muuttopäivä' in text:
                moving_date_ = (text.split('Toivottu muuttopäivä')[1].strip())[3:12].strip().split(',')[0]
                if moving_date_ == 'nkohta av': 
                    moving_date_ = ''
            if 'Komissio' in text:
                commission_ = (text.split('Komissio')[1].strip())[:3].strip()

        rnd = random.randint(10,20)
        time.sleep(rnd)
        # Sending offer by cookies
        response = requests.post(url, headers=headers, data=payload, cookies=cookies)

        # Response check
        if response.status_code == 200:
            status_ = "Registered successfully"
        else:
            status_ = "Registration error: ", response.status_code

        local_time = time.localtime()
        current_date = time.strftime("%Y-%m-%d", local_time)
        current_time = time.strftime("%H:%M:%S", local_time)

        new_data = [data_ids[i], current_date, current_time, from_, where_, moving_date_, commission_, status_]
        append_data_to_excel(file_name, new_data)
        print(current_time, " ---> ", url)
    
    rnd = random.randint(5,10)
    time.sleep(rnd * 60)